In [1]:
import sys
sys.path.append('..')

import torch
import numpy as np
from torch.utils.data import DataLoader

from evaluate.load.helpers import *
from evaluate.load.dataset import *
from evaluate.pipeline.model import *
from evaluate.pipeline.trainer import *
from evaluate.pipeline.embedding import *

In [2]:
def get_param(mode="paper",isHashEmbedding=True):
    return {"use_hash_embeddings": isHashEmbedding,
            "hashing_trick": True,
            "ngram_range": (1,9),
            "nFeaturesRange": (4,100),
            "embedding_size": 20,
            "num_buckets": 10**6,
            "max_features": 10**7,
            "max_epochs": 100,
            "num_hash_functions": 2,
            "hidden": 50,
            "seed": 123,
            "batch_size": 32,
            "masking":True,
            "append_weight":False,
            "validation_size":0.05,
            "patience":10,
            'cuda':True,
            'num_workers':16}
            
param = get_param()
seed = param['seed']
max_features = param['max_features']
ngram_range = param['ngram_range']
use_hash_embeddings = param['use_hash_embeddings']
isHashingTrick = param['hashing_trick']
num_buckets = param['num_buckets']
embedding_size = param['embedding_size']
max_epochs = param['max_epochs']
masking = param['masking']
append_weight = param['append_weight']
patience = param['patience']
num_hash_functions = param['num_hash_functions']
batch_size = param['batch_size']
validation_size = param['validation_size']
nFeaturesRange = param['nFeaturesRange']
isCuda = param['cuda']
num_workers = param['num_workers']

In [3]:
%%time

np.random.seed(seed)
torch.manual_seed(seed)

train = AgNews(nFeaturesRange=nFeaturesRange,maxFeatures=max_features-1,train=True,ngram_range=ngram_range,isHashingTrick=isHashingTrick,seed=seed)
test = AgNews(nFeaturesRange=nFeaturesRange,maxFeatures=max_features-1,train=False,ngram_range=ngram_range,isHashingTrick=isHashingTrick,seed=seed)

CPU times: user 32.6 s, sys: 658 ms, total: 33.3 s
Wall time: 33.2 s


In [4]:
#train, valid = train_valid_load(train,validSize=0.1,isShuffle=True,seed=123,batch_size=batchSize)
test = DataLoader(dataset=test,batch_size=batch_size,shuffle=True)

In [ ]:
isHash=True
isCuda=True

num_classes = len(train.classes)
model = ModelNoDict(max_features,embedding_size,num_classes,isHash=isHash,append_weight=True,num_buckets=num_buckets)
if isCuda:
    model.cuda()
    if isHash:
        model.embedding.hashes = model.embedding.hashes.cuda()
trainer = Trainer(model,seed=seed,isCuda=isCuda)
callbacks = [EarlyStopping(patience=patience)]

Using CUDA


In [ ]:
%%time
trainer(train,callbacks=callbacks,validSize=validation_size,num_workers=num_workers)

Num parameters in model: 40000092
Train on 3563 samples, validate on 188 samples


In [ ]:
isHash=False
isCuda=True

num_classes = len(train.classes)
model = ModelNoDict(max_features,embedding_size,num_classes,isHash=isHash,append_weight=True,num_buckets=num_buckets)
if isCuda:
    model.cuda()
    if isHash:
        model.embedding.hashes = model.embedding.hashes.cuda()
trainer = Trainer(model,seed=seed,isCuda=isCuda)
callbacks = [EarlyStopping(patience=patience)]

In [ ]:
%%time
trainer(train,callbacks=callbacks,validSize=validation_size,num_workers=num_workers)

In [ ]:
isHash=True
isCuda=False

num_classes = len(train.classes)
model = ModelNoDict(max_features,embedding_size,num_classes,isHash=isHash,append_weight=True,num_buckets=num_buckets)
if isCuda:
    model.cuda()
    if isHash:
        model.embedding.hashes = model.embedding.hashes.cuda()
trainer = Trainer(model,seed=seed,isCuda=isCuda)
callbacks = [EarlyStopping(patience=patience)]

In [ ]:
%%time
trainer(train,callbacks=callbacks,validSize=validation_size,num_workers=num_workers)

In [55]:
%%time
trainer(train,callbacks=callbacks,validSize=0.1)

Num parameters in model: 2000084
Train on 3375 samples, validate on 375 samples
Epoch: 0. Loss: 0.16543151438236237. Acc: 0.9149166666666667.
Epoch: 1. Loss: 0.2636685371398926. Acc: 0.9113333333333333.
Epoch: 2. Loss: 0.11431225389242172. Acc: 0.90375.
Stoping at epoch:2 with patience:2. Best:0.9149166666666667.
CPU times: user 5min 55s, sys: 1.13 s, total: 5min 56s
Wall time: 5min 57s
